In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from skorch import NeuralNetRegressor
import geopandas

In [2]:
# read geopandas dataframe
df = geopandas.read_file(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\\2.merge_data\intermediate\output14.geojson')


In [3]:
#drop geometry column
df = df.drop(columns=['geometry'])

In [4]:


# drop columns with more than 50% missing values
df = df.dropna(thresh=0.5*len(df), axis=1)

# drop columns with only one unique value
df = df.loc[:,df.apply(pd.Series.nunique) != 1]

# drop columns with all missing values
df = df.dropna(axis=1, how='all')

# drop columns with all zero values
df = df.loc[:, (df != 0).any(axis=0)]


# drop all missing values
df = df.dropna()

In [5]:
X = df[['radiation', 'soil_temperature', 'temperature',
       'soil_quality', 'battles', 'explosions', 'violence',
       'fatalities_battles', 'fatalities_explosions', 'fatalities_violence','estimated_population','elevation',
       'soil', 'price', 'GOSIF_GPP', 'chirps_rainfall', 'GOSIF_GPP_SD']].values.astype(np.float32)
y = df['phase3_worse_percentage_manual'].values.astype(np.float32)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
from skorch.callbacks import Callback
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(X)
X = imputer.transform(X)


class GradClipCallback(Callback):
    def __init__(self, max_norm):
        self.max_norm = max_norm

    def on_backward_end(self, net, named_parameters, Xi, yi, training, **kwargs):
        torch.nn.utils.clip_grad_norm_(net.module.parameters(), self.max_norm)

class SimpleNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(64, 128)
        self.dropout2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(128, 64)
        self.dropout3 = nn.Dropout(0.2)
        self.fc4 = nn.Linear(64, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = F.relu(self.fc3(x))
        x = self.dropout3(x)
        x = self.fc4(x)
        return x


In [8]:

input_dim = X.shape[1]
output_dim = 1

# Scale the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = SimpleNN(input_dim, output_dim)

net = NeuralNetRegressor(
    model,
    criterion=nn.MSELoss,
    optimizer=optim.Adam,
    lr=0.005,
    max_epochs=100,
    batch_size=32,
    train_split=None,
    verbose=0,
    callbacks=[
        ('grad_clip', GradClipCallback(max_norm=1.0)),
    ],
)

net.fit(X_train_scaled, y_train.reshape(-1, 1))
y_pred = net.predict(X_test_scaled)

In [9]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Calculate metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print metrics
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("Mean Absolute Error:", mae)
print("R-squared:", r2)


Mean Squared Error: 0.030065697
Root Mean Squared Error: 0.17339462
Mean Absolute Error: 0.11100757
R-squared: 0.8325700050699856


In [12]:
from sklearn.inspection import permutation_importance

# Calculate permutation importance
result = permutation_importance(net, X_test_scaled, y_test, n_repeats=10, random_state=42)
feature_names = ['radiation', 'soil_temperature', 'temperature',
       'soil_quality', 'battles', 'explosions', 'violence',
       'fatalities_battles', 'fatalities_explosions', 'fatalities_violence','estimated_population','elevation',
       'soil', 'price', 'GOSIF_GPP', 'chirps_rainfall', 'GOSIF_GPP_SD']
# Print the feature importances
for feature, importance in zip(feature_names, result.importances_mean):
    print(f"{feature}: {importance}")

radiation: 0.18408618751806055
soil_temperature: 0.16908434428689073
temperature: 0.2673031102186404
soil_quality: 0.18819787264836293
battles: 0.12645501026868164
explosions: 0.02038636719935284
violence: 0.06168445083977799
fatalities_battles: 0.019049021547110044
fatalities_explosions: 0.010488315492616872
fatalities_violence: 0.09230784863677434
estimated_population: 0.202890828080946
elevation: 0.20202030018790734
soil: 0.2232137810348805
price: 0.5133846882525124
GOSIF_GPP: 0.31987914493422764
chirps_rainfall: 0.2980079852522823
GOSIF_GPP_SD: 0.09249644479490442


In [17]:
import joblib

# Save the model and scaler to disk
joblib.dump(net, "regressor.pkl")
joblib.dump(scaler, "scaler.pkl")


['scaler.pkl']